# Importing Modules

In [1]:
# Data wrangling libraries
import pandas as pd
import numpy as np
import cudf # GPU version of pandas
from numba import cuda
import os
import datashader as ds

# ML Libraries
from cuml import KMeans # GPU version of scikit-learn

# Plotting libraries
import multiprocessing as mp

# wrapper functions
from resultHelperClass import Result
from open_source_algo.cvxEDA import wrapperCVXEDA

In [27]:
%env LD_LIBRARY_PATH /usr/local/cuda-11.8/lib64

env: LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64


In [28]:
%env LD_LIBRARY_PATH

'/usr/local/cuda-11.8/lib64'

# Scary Movie Data

In [2]:
scary_movie_session_gpu = cudf.read_csv("resources/csv/default_exp_Session1_Shimmer_92E8_Calibrated_PC_Session2.csv", delimiter="\t", header=1)

In [3]:
scary_movie_session_gpu.drop(columns=scary_movie_session_gpu.columns[-1], axis=1, inplace=True)

In [4]:
scary_movie_session_gpu.drop(index=0, axis=0, inplace=True)
scary_movie_session_gpu.reset_index(drop=True, inplace=True)

In [5]:
scary_movie_session_gpu = scary_movie_session_gpu.astype("float64")

In [6]:
scary_movie_session_gpu = cudf.from_pandas(scary_movie_session_gpu.to_pandas().apply(lambda x: x - scary_movie_session_gpu.loc[0, scary_movie_session_gpu.columns[0]]))

In [14]:
%%capture
Fs = 150  # the frequency of the shimmer gsr+
processor = 1
pool = mp.Pool(mp.cpu_count())
result = Result()
# slow convex optimization for Electro-dermal activity (cvxEDA)
pool.apply_async(wrapperCVXEDA, [scary_movie_session_gpu.Shimmer_92E8_GSR_Skin_Conductance_CAL.to_numpy(), 1 / Fs], callback=result.update_value)

In [7]:
cvs = ds.Canvas(plot_height=650, plot_width=800)

In [8]:
cvs.line(source=scary_movie_session_gpu, x=scary_movie_session_gpu.columns[0], y=scary_movie_session_gpu.columns[1])

<xarray.DataArray (Shimmer_92E8_GSR_Skin_Conductance_CAL: 650,
                   Shimmer_92E8_TimestampSync_Unix_CAL: 800)>
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
Coordinates:
  * Shimmer_92E8_TimestampSync_Unix_CAL    (Shimmer_92E8_TimestampSync_Unix_CAL) float64 ...
  * Shimmer_92E8_GSR_Skin_Conductance_CAL  (Shimmer_92E8_GSR_Skin_Conductance_CAL) float64 ...

In [15]:
res = []

In [18]:
if result.is_done():
    """
        r: phasic component
        p: sparse SMNA driver of phasic component
        t: tonic component
        l: coefficients of tonic spline
        d: offset and slope of the linear drift term
        e: model residuals
        obj: value of objective function being minimized (eq 15 of paper)
    """
    res = result.get_value()
else:
    print("Not yet!")

Not yet!


In [ ]:
scary_movie_session_gpu["phasic"] = res[0]

In [ ]:
kmeans = KMeans(n_clusters=2, n_init="auto")

In [ ]:
scary_movie_session_gpu["label"] = kmeans.fit_predict(scary_movie_session_gpu[["phasic"]])

In [ ]:
scary_movie_session_gpu.label = scary_movie_session_gpu.label.apply(bool)